Functional model with only time travel cost</br>
231030

In [344]:
from pyscipopt import Model, quicksum
import numpy as np
from itertools import product
import json

In [345]:
#Importing constant variables
with open('./EVRP to LP/input_variables.json', 'r') as file:
    input_variables = json.load(file)

# print(input_variables["Ctt"])

In [346]:
def instanSacy(n_customers, n_vehicles, max_distance):
    N = list(np.arange(n_customers+1)) #Set of customers
    A = [(i,j) for i,j in product(N,N) if i!=j]
    
    # demand = {i:450 if i!=0 else 0 for i in N} #Demand per customer
    demand = {i:int(np.random.randint(100, 300, 1)[0]) for i in N} #Demand per customer

    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    #Time cost as a function of distance and avg. speed
    distance =  {(i,j):int(np.random.randint(50, max_distance, 25)[0]) for i,j in A}
    time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A}
    # time_cost = {(i,j):int(np.random.randint(1, max_time_cost, 1)[0]) for i,j in A} #Travel time cost per arc

    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i} #Energy cost per arc
    

    return N, A, demand, M, load_capacity, time_cost, distance

In [347]:
def sacystation(N, A, demand, M, load_capacity, time_cost):
    """transp -- model for solving the transportation problem
    Parameters:
        N - set of customers
        A - set of arcs (i,j)
        M - set of vehicles
        energy_cost[i,j] - energy transportation cost on arc (i,j)
        demand[i] - demand of customer i
        load_capacity[m] - capacity of vehicle m
        time_cost - Travel time per arc(i,j)

    ==================================
    Problems:
    Time constraint not respecting number of cars 
    """
    
    model = Model("sacystation")

    #Binary decision variable
    x = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
        
    #Load Variable
    L = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        L[i,j,m] = model.addVar(vtype="C", name=f"L{(i,j,m)}")
    
    '''
    Constraints the number of times that vehicle m
    can leave or returns to the depot
    '''
    for m in M:
        if len(N) > len(M):
            model.addCons(quicksum(x[0,j,m] for j in N if j!= 0) >= 1)
        else:
            model.addCons(quicksum(x[0,j,m] for j in N if j!= 0) <= 1)
                       
    '''
    Each customer is visited only once
    '''        
    for j in N:
        if j!=0:
            model.addCons(quicksum(x[0,j,m] for m in M) == 1)

    '''
    ACTS like a flow conservation constraint:
    Enforces that the vehicle must return to the depot to load
    the demand for the next vertex
    '''
    # for j in N:
    #     model.addCons((quicksum(x[0,j,m] for m in M if j!= 0)) 
    #               == (quicksum(x[j,0,m] for m in M if j!= 0))) 
        
    '''
    Enforces that the vehicle must return to the depot from the
    vertex which it delivered 
    '''    
    for m in M:
        for j in N:
            if j!= 0:
                model.addCons(x[0,j,m] == x[j,0,m])
       
    '''
    Demand constraint
    Vehicle departs with only demand[i] load, then returns to depot
    '''
    for m in M:
        model.addCons(quicksum(L[0,j,m] for j in N if j!= 0) == demand[j] * x[0,j,m])
        model.addCons(quicksum(L[j,0,m] for j in N if j!= 0) == 0)
           
    #Time limit constraint
    for m in M:
        model.addCons(quicksum(time_cost[i,j]*x[i,j,m] for i,j in A if i!=j) <= 8)
    
    # Objective - ONLY TRAVEL TIME  
    travel_time_cost = quicksum(input_variables["Ctt"]*time_cost[i,j]*x[i,j,m] for (i,j) in A for m in M)
    model.setObjective(travel_time_cost, "minimize")

    model.optimize()

    model.data = x
    
    return model, travel_time_cost


In [348]:
if __name__ == "__main__":
    N, A, demand, M, load_capacity, time_cost, distance = instanSacy(1,2,500)
    model, travel_time_cost = sacystation(N, A, demand, M, load_capacity, time_cost)    
    model.optimize()
    
    EPS = 1.e-6  # a small tolerance
    x = model.data  # model.data contains the decision variables

    # Time cost
    # cost = sum(time_cost[i,j]*x[i,j,m] for i,j in A if i!=j)
    # values = cost.terms.values()
    # print("Total time:", sum(values))
    print(f'Travel time ={(travel_time_cost.terms.values())}')
    print(f'{time_cost=}')
    print(f'Total time cost={sum(time_cost.values())}')
    print(f'{distance=}')
    # print(N,M, sep='\n')
    
    if model.getStatus() == "optimal":
        print("Optimal value:", model.getObjVal())
    else:
        print("Problem could not be solved to optimality")      

    for (i, j, m) in x:
        if model.getVal(x[i, j, m]) > 0.5:
            print(f"x({i}, {j}, {m}) = {model.getVal(x[i, j, m])}")
        # else:
        #     print(f"x[{i}, {j}, {m}] = 0")
    


Travel time =dict_values([0.0, 38.56, 38.56, 12.0, 12.0])
time_cost={(0, 1): 2.41, (1, 0): 0.75}
Total time cost=3.16
distance={(0, 1): 193, (1, 0): 60}
Optimal value: 50.56
x(0, 1, 1) = 1.0
x(1, 0, 1) = 1.0


In [349]:
model.writeProblem(r"D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp")
model.freeProb()

wrote problem to file D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp


In [350]:
model = Model("sacystation")

def instanSacy(n_customers, n_vehicles, max_time_cost):
    N = list(np.arange(1,n_customers+1)) #Set of customers
    A = [(i,j) for i,j in product(N,N) if i!=j]

    demand = {i:450 if i!=0 else 0 for i in N} #Demand per customer
    # demand = {i:int(np.random.randint(100, 300, 1)[0]) for i in A} #Demand per customer

    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:9073 for m in M}
                    #{m:load_capacity for m in M} #Load_capacity per vehicle

    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i}#Energy cost per arc

    time_cost = {(i,j):int(np.random.randint(1, max_time_cost, 1)[0]) for i,j in A}#Travel time cost per arc

    return N, A, demand, M, load_capacity, time_cost

N, A, demand, M, load_capacity, time_cost = instanSacy(2,1,6)

model = Model("sacystation")

#Creating binary decision variable
x = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M ]:
    x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
    
# if (0,1) in x:
#     print(True)
# else:
#     print(False)
    
print(N, A, M,x, sep='\n')
# still need to add the constraints
#Restrics number of m leaving the depot
for m in M:
    model.addCons(quicksum(x[0,j,m] for j in N if (j,m) in x) == 1)
    # model.addCons(quicksum(x[i,0,m] for i in N if (i,m) in x) == 1)
D = N 
D.insert(0,0)
print(D, type(D))
print([(i,j) for i,j in product(D,D)])




[1, 2]
[(1, 2), (2, 1)]
[1]
{(1, 2, 1): x(1, 2, 1), (2, 1, 1): x(2, 1, 1)}
[0, 1, 2] <class 'list'>
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
